In [5]:
import toupcam

In [35]:
a = toupcam.Toupcam.EnumV2()

In [36]:
if len(a) > 0:
    print('{}: flag = {:#x}, preview = {}, still = {}'.format(a[0].displayname, a[0].model.flag, a[0].model.preview, a[0].model.still))
    for r in a[0].model.res:
        print('\t = [{} x {}]'.format(r.width, r.height))   #输出分辨率选项

BigEye4200KME: flag = 0x20087182059, preview = 2, still = 0
	 = [2048 x 2048]
	 = [1024 x 1024]


In [37]:
hcam = toupcam.Toupcam.Open(a[0].id)   #打开第一个相机
#hcam.put_eSize(1)   #更改分辨率
width, height = hcam.get_Size()
bufsize = toupcam.TDIBWIDTHBYTES(width * 48) * height  #图片缓存RGB48,字节数
hcam.put_Option(toupcam.TOUPCAM_OPTION_RGB, 1)    #RGB48

print('image size: {} x {}, bufsize = {}'.format(width, height, bufsize))  #显示相机分辨率和图片缓存
#显示可调的曝光时间和增益范围
uimin, uimax, uidef = hcam.get_ExpTimeRange()
print(f"uimin:{uimin}, uimax:{uimax} uidef：{uidef}")
usmin, usmax, usdef = hcam.get_ExpoAGainRange()
print(f"usmin:{usmin}, usmax:{usmax} usdef：{usdef}")

buf = bytes(bufsize)  #转换为可拉取的缓存格式
#hcam.StartPullModeWithCallback(self.cameraCallback, self)   #拉取图片


image size: 2048 x 2048, bufsize = 25165824
uimin:21, uimax:60000000 uidef：10000
usmin:100, usmax:800 usdef：100


In [38]:
#切换曝光增益和曝光时间
hcam.put_ExpoTime(10000)
hcam.put_ExpoAGain(100)
ExpoTime = hcam.get_ExpoTime()
ExpoAGain = hcam.get_ExpoAGain()

usmin, usmax, usdef = hcam.get_ExpoAGainRange()
print(f"ExpoTime:{ExpoTime}, ExpoAGain:{ExpoAGain}")

ExpoTime:10000, ExpoAGain:100


In [39]:
hcam.Close()
hcam = None
buf = None

In [1]:
import toupcam
import numpy as np
from PyQt5.QtGui import QPixmap, QImage
from PIL import Image
import tifffile as tiff
import os
import threading
import time
import re
import openpyxl
import json
from scipy.stats import linregress

class App:
    def __init__(self):
        self.hcam = None
        self.buf = None
        self.total = 0
        self.width =  0
        self.height = 0
        self.ExpoTime = None
        self.ExpoAGain = None
        self.BLE = None
        self.BLEpath = None
        self.FPN = None

# the vast majority of callbacks come from toupcam.dll/so/dylib internal threads
    @staticmethod
    def cameraCallback(nEvent, ctx):
        if nEvent == toupcam.TOUPCAM_EVENT_IMAGE:

            ctx.CameraCallback(nEvent)

    def CameraCallback(self, nEvent):
        if nEvent == toupcam.TOUPCAM_EVENT_IMAGE:
            try:
                ExpoTime = self.hcam.get_ExpoTime()  #获取当前图片的曝光时间和增益系数
                ExpoAGain = self.hcam.get_ExpoAGain()
                print(f"{self.total+1}:{ExpoTime}-{ ExpoAGain }")
                
                self.hcam.PullImageV3(self.buf, 0, 16, 0, None)
                self.total += 1
                current_thread_id = threading.current_thread().ident
                print(f'pull image ok, total = {self.total}   {current_thread_id}')
                
            except toupcam.HRESULTException as ex:
                print('pull image failed, hr=0x{:x}'.format(ex.hr & 0xffffffff))

            else:
                if self.BLE == 1:
                    self.onBLEprocess() 
                elif self.FPN == 1:
                    self.onFPNprocess()
        else:
            print('event callback: {}'.format(nEvent))

    def onBLEprocess(self):
        ExpoTime = self.hcam.get_ExpoTime()  #获取当前图片的曝光时间和增益系数
        ExpoAGain = self.hcam.get_ExpoAGain()
        print(f"{self.total}:{ExpoTime}-{ ExpoAGain }")
        if ExpoTime == self.ExpoTime and ExpoAGain == self.ExpoAGain:
            image = QImage(self.buf, self.width, self.height, QImage.Format_Grayscale16)  
            ptr = image.constBits()
            image_arr = np.frombuffer(ptr.asstring(2048*2048*2), dtype=np.uint16).reshape(self.height, -1)* 16

            os.makedirs(self.BLEpath, exist_ok=True)
            file_name = f"{self.BLEpath}/{ExpoTime}_{ExpoAGain}.tif"
            tiff.imwrite(file_name ,image_arr)
            
            #系数变化逻辑
            ExpoAGain = ExpoAGain + 100
            if ExpoAGain == 900:
                ExpoTime = ExpoTime*2
                ExpoAGain = 100
            
            self.hcam.put_ExpoTime(ExpoTime)
            self.hcam.put_ExpoAGain(ExpoAGain)

        else:  #下一次再读取缓存，保证设置刷新
            self.ExpoTime = ExpoTime
            self.ExpoAGain = ExpoAGain

    def onFPNprocess(self):
        return
        

    def BLEpreprocess(self):
        self.BLEpath = "camera/autoBLE"
        self.hcam.put_ExpoTime(100000)  #初始曝光时间
        self.hcam.put_ExpoAGain(100)       #初始增益系数
        self.ExpoTime = self.hcam.get_ExpoTime()  #获取当前图片的曝光时间和增益系数
        self.ExpoAGain = self.hcam.get_ExpoAGain()
        print(f"initial:{self.ExpoTime}-{self.ExpoAGain }")
        #获取图片到指定文件夹
        self.BLE = 1;#处理黑电平模式
        if self.BLE == 1:
            while self.ExpoTime<2000000:
                time.sleep(1)
        self.BLE = None #关闭处理
        #处理图片数据存到Excel
        process_BLEimages(self.BLEpath)
        fit_linear_model_to_data_and_save('output.xlsx')
    

        

    def run(self):
        a = toupcam.Toupcam.EnumV2()
        if len(a) > 0:
            print('{}: flag = {:#x}, preview = {}, still = {}'.format(a[0].displayname, a[0].model.flag, a[0].model.preview, a[0].model.still))
            for r in a[0].model.res:
                print('\t = [{} x {}]'.format(r.width, r.height))   #输出分辨率选项
            self.hcam = toupcam.Toupcam.Open(a[0].id)   #打开第一个相机
            if self.hcam:
                try:
                    self.width, self.height = self.hcam.get_Size()
                    bufsize = self.width * self.height * 2
                    self.hcam.put_Option(toupcam.TOUPCAM_OPTION_BYTEORDER, 0)   #Qimage use RGB byte order
                    self.hcam.put_Option(toupcam.TOUPCAM_OPTION_RAW, 0)   #Qimage use RGB byte order
                    self.hcam.put_Option(toupcam.TOUPCAM_OPTION_BITDEPTH, 1)    #channel depth 12bit
                    self.hcam.put_Option(toupcam.TOUPCAM_OPTION_RGB, 4)         #mode 16 gray
                    self.hcam.put_Option(toupcam.TOUPCAM_OPTION_UPSIDE_DOWN, 0)
                    self.hcam.put_Option(toupcam.TOUPCAM_OPTION_MULTITHREAD,0)
                    self.hcam.put_AutoExpoEnable(0)   #自动曝光关
                    self.hcam.put_ExpoTime(100000)  #
                    self.hcam.put_ExpoAGain(100)      

                    print('image size: {} x {}, bufsize = {}'.format(self.width, self.height, bufsize))
                    self.buf = bytes(bufsize)
                    if self.buf:
                        try:
                            self.hcam.StartPullModeWithCallback(self.cameraCallback, self)
                        except toupcam.HRESULTException as ex:
                            print('failed to start camera, hr=0x{:x}'.format(ex.hr & 0xffffffff))

                    self.BLEpreprocess();#预处理获取各个ISO下的BLE参数
                    #input('press ENTER to exit')
       
                finally:
                    self.hcam.Close()   #关闭相机
                    self.hcam = None
                    self.buf = None
            else:
                print('failed to open camera')
        else:
            print('no camera found')


def calculate_average_pixel_value(image_path):
    """ 计算图片像素的平均值 """
    with Image.open(image_path) as img:
        img_array = np.array(img)
        return np.mean(img_array)

def get_coordinates_from_filename(filename):
    """ 从文件名中提取坐标 """
    match = re.match(r'(\d+)_(\d+)', filename)
    if match:
        return int(match.group(1)), int(match.group(2))
    return None, None

def process_BLEimages(folder_path):
    """ 处理指定文件夹中的所有图片 """
    wb = openpyxl.Workbook()
    ws = wb.active

    # 收集所有的曝光时间和增益系数
    exposures = set()
    gains = set()
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp')):
            exposure, gain = get_coordinates_from_filename(filename)
            if exposure is not None and gain is not None:
                exposures.add(exposure)
                gains.add(gain)

    # 排序
    sorted_exposures = sorted(exposures)
    sorted_gains = sorted(gains)

    # 在Excel中创建标题行和列
    for i, exposure in enumerate(sorted_exposures, start=2):
        ws.cell(row=1, column=i, value=exposure)
    for i, gain in enumerate(sorted_gains, start=2):
        ws.cell(row=i, column=1, value=gain)

    # 填充数据
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff', '.bmp')):
            exposure, gain = get_coordinates_from_filename(filename)
            if exposure is not None and gain is not None:
                avg_pixel_value = calculate_average_pixel_value(os.path.join(folder_path, filename))
                exposure_index = sorted_exposures.index(exposure) + 2
                gain_index = sorted_gains.index(gain) + 2
                ws.cell(row=gain_index, column=exposure_index, value=avg_pixel_value)

    wb.save('output.xlsx')

#拟合BLE每个ISO下的线性函数，并存成Excel和json
def fit_linear_model_to_data_and_save(excel_file):
    wb = openpyxl.load_workbook(excel_file)
    ws = wb.active
    exposure_times = [cell.value for cell in ws[1] if cell.value is not None]
    print(exposure_times)

    # 准备存储斜率和截距
    last_column = ws.max_column
    ws.cell(row=1, column=last_column + 1, value="Slope (k)")
    ws.cell(row=1, column=last_column + 2, value="Intercept (b)")

    iso_params = {}

    for row_index, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=1), start=2):
        gain = row[0].value
        if gain is not None:
            pixel_values = [ws.cell(row=row_index, column=i+2).value for i in range(len(exposure_times))]
            slope, intercept, _, _, _ = linregress(exposure_times, pixel_values)
            ws.cell(row=row_index, column=last_column + 1, value=slope)
            ws.cell(row=row_index, column=last_column + 2, value=intercept)
            iso_params[gain] = {'slope': slope, 'intercept': intercept}

    # 保存修改后的Excel文件
    wb.save('fitted_output.xlsx')

    # 保存为JSON文件
    with open('iso_params.json', 'w') as f:
        json.dump(iso_params, f, indent=4, sort_keys=True)

In [2]:
app = App()
app.run()

BigEye4200KME: flag = 0x20087182059, preview = 2, still = 0
	 = [2048 x 2048]
	 = [1024 x 1024]
image size: 2048 x 2048, bufsize = 8388608
initial:100000-100
1:100000-100
pull image ok, total = 1   11428
1:100000-100
2:100000-200
pull image ok, total = 2   11428
2:100000-200
3:100000-200
pull image ok, total = 3   11428
3:100000-200
4:100000-300
pull image ok, total = 4   11428
4:100000-300
5:100000-300
pull image ok, total = 5   11428
5:100000-300
6:100000-400
pull image ok, total = 6   11428
6:100000-400
7:100000-400
pull image ok, total = 7   11428
7:100000-400
8:100000-500
pull image ok, total = 8   11428
8:100000-500
9:100000-500
pull image ok, total = 9   11428
9:100000-500
10:100000-600
pull image ok, total = 10   11428
10:100000-600
11:100000-600
pull image ok, total = 11   11428
11:100000-600
12:100000-700
pull image ok, total = 12   11428
12:100000-700
13:100000-700
pull image ok, total = 13   11428
13:100000-700
14:100000-800
pull image ok, total = 14   11428
14:100000-800
1